In [2]:
import numpy as np
import pandas as pd

# 60. 単語ベクトルの読み込みと表示Permalink

In [3]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)


In [5]:
model['United_States'].shape

(300,)

In [10]:
model['US'].shape

(300,)

In [16]:
model['US'].reshape(-1).shape

(300,)

In [18]:
np.array([1, 2, 3]).shape

(3,)

# 61. 単語の類似度

In [26]:
model.similarity('United_States', 'U.S.')

0.73107743

# 62. 類似度の高い単語10件

In [30]:
model.most_similar('United_States')

[('Unites_States', 0.7877248525619507),
 ('Untied_States', 0.7541370391845703),
 ('United_Sates', 0.74007248878479),
 ('U.S.', 0.7310774326324463),
 ('theUnited_States', 0.6404393911361694),
 ('America', 0.6178410053253174),
 ('UnitedStates', 0.6167312264442444),
 ('Europe', 0.6132988929748535),
 ('countries', 0.6044804453849792),
 ('Canada', 0.6019070148468018)]

# 63. 加法構成性によるアナロジー

In [34]:
vec = (model['Spain'] - model['Madrid'] + model['Athens'])
model.most_similar(positive=['Spain', 'Athens'], negative=['Madrid'], topn=10)

[('Greece', 0.6898480653762817),
 ('Aristeidis_Grigoriadis', 0.5606849193572998),
 ('Ioannis_Drymonakos', 0.555290937423706),
 ('Greeks', 0.5450686812400818),
 ('Ioannis_Christou', 0.5400862693786621),
 ('Hrysopiyi_Devetzi', 0.5248445272445679),
 ('Heraklio', 0.5207759141921997),
 ('Athens_Greece', 0.516880989074707),
 ('Lithuania', 0.5166865587234497),
 ('Iraklion', 0.5146791338920593)]

In [42]:
!wget http://download.tensorflow.org/data/questions-words.txt
!mv questions-words.txt ../data/

--2020-09-26 01:29:01--  http://download.tensorflow.org/data/questions-words.txt
Resolving download.tensorflow.org... 2404:6800:4004:81f::2010, 216.58.220.144
Connecting to download.tensorflow.org|2404:6800:4004:81f::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘questions-words.txt.3’

questions-words.txt 100%[===================>] 589.80K  1.18MB/s    in 0.5s    

2020-09-26 01:29:01 (1.18 MB/s) - ‘questions-words.txt.3’ saved [603955/603955]



In [43]:
!head -10 ../data/questions-words.txt

: capital-common-countries
Athens Greece Baghdad Iraq
Athens Greece Bangkok Thailand
Athens Greece Beijing China
Athens Greece Berlin Germany
Athens Greece Bern Switzerland
Athens Greece Cairo Egypt
Athens Greece Canberra Australia
Athens Greece Hanoi Vietnam
Athens Greece Havana Cuba


In [46]:
with open('../data/questions-words.txt', "r") as fin, open('../output/section7/qustion-words-add.txt', 'w') as fout:
    for line in fin:
        splitted = line.split(" ")
        if splitted[0] == ":":
            category = splitted[1]
        else:
            factor1 = splitted[0]
            factor2 = splitted[1]
            factor3 = splitted[2]
            factor4 = splitted[3]
            most_similars = model.most_similar(positive=[factor2, factor3], negative=[factor1], topn=10)
            most_similar = most_similars[0]
            word = most_similar[0]
            score = most_similar[1]
            fout.write(''.join([category] + splitted + [word, str(score) + '\n']))
            
        
            
        

KeyboardInterrupt: 

In [28]:
# from sklearn.metrics.pairwise import cosine_similarity

# cosine_similarity(model['United_States'], model['U.S.'])

ValueError: Expected 2D array, got 1D array instead:
array=[-3.61328125e-02 -4.83398438e-02  2.35351562e-01  1.74804688e-01
 -1.46484375e-01 -7.42187500e-02 -1.01562500e-01 -7.71484375e-02
  1.09375000e-01 -5.71289062e-02 -1.48437500e-01 -6.00585938e-02
  1.74804688e-01 -7.71484375e-02  2.58789062e-02 -7.66601562e-02
 -3.80859375e-02  1.35742188e-01  3.75976562e-02 -4.19921875e-02
 -3.56445312e-02  5.34667969e-02  3.68118286e-04 -1.66992188e-01
 -1.17187500e-01  1.41601562e-01 -1.69921875e-01 -6.49414062e-02
 -1.66992188e-01  1.00585938e-01  1.15722656e-01 -2.18750000e-01
 -9.86328125e-02 -2.56347656e-02  1.23046875e-01 -3.54003906e-02
 -1.58203125e-01 -1.60156250e-01  2.94189453e-02  8.15429688e-02
  6.88476562e-02  1.87500000e-01  6.49414062e-02  1.15234375e-01
 -2.27050781e-02  3.32031250e-01 -3.27148438e-02  1.77734375e-01
 -2.08007812e-01  4.54101562e-02 -1.23901367e-02  1.19628906e-01
  7.44628906e-03 -9.03320312e-03  1.14257812e-01  1.69921875e-01
 -2.38281250e-01 -2.79541016e-0